**TUGAS KAPITA SELEKTA STATISTIKA**

Kelompok Murid Prof:

1. Kenneth Holivianto   \ 6162001018
2. Jason Hinardi        \ 6162001022
3. Bryan Ernestin       \ 6162001097
4. Leonardo Alindra     \ 6162001111

# **Pemrosesan Data**

In [ ]:
# Instalasi pyspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=5a075da50a926a6f063acff840afea37e8c4433d92aecba0d83fcea947d03edd
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
# Import library yang digunakan
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, min, max, stddev, desc
import matplotlib.pyplot as plt

In [ ]:
# Membuat spark session
spark = SparkSession.builder.appName("Analisis ISPU").getOrCreate()

In [ ]:
# Membaca dan mengekstrak data
df = spark.read.csv("/content/Data_ISPU.csv", header=True)

df.show(10)

+---------+------------------+----+---+---+---+---+---+--------+--------+
|  tanggal|           stasiun|pm10|so2| co| o3|no2|max|critical|categori|
+---------+------------------+----+---+---+---+---+---+--------+--------+
| 1/1/2020|DKI1 (Bunderan HI)|  30| 20| 10| 32|  9| 32|      O3|    BAIK|
| 2/1/2020|DKI1 (Bunderan HI)|  27| 22| 12| 29|  8| 29|      O3|    BAIK|
| 3/1/2020|DKI1 (Bunderan HI)|  39| 22| 14| 32| 10| 39|    PM10|    BAIK|
| 4/1/2020|DKI1 (Bunderan HI)|  34| 22| 14| 38| 10| 38|      O3|    BAIK|
| 5/1/2020|DKI1 (Bunderan HI)|  35| 22| 12| 31|  9| 35|    PM10|    BAIK|
| 6/1/2020|DKI1 (Bunderan HI)|  46| 23| 16| 32|  9| 46|    PM10|    BAIK|
| 7/1/2020|DKI1 (Bunderan HI)|  37| 23| 26| 33| 11| 37|    PM10|    BAIK|
| 8/1/2020|DKI1 (Bunderan HI)|  41| 26| 20| 30| 11| 41|    PM10|    BAIK|
| 9/1/2020|DKI1 (Bunderan HI)|  52| 23| 29| 24| 12| 52|    PM10|  SEDANG|
|10/1/2020|DKI1 (Bunderan HI)|  24| 24| 18| 25|  8| 25|      O3|    BAIK|
+---------+------------------+----+---

In [ ]:
# Mengetahui banyaknya baris pada data
df.count()

1830

In [ ]:
# Menghapus barisan data yang tidak lengkap
df = df.dropna()

In [ ]:
# Melihat kembali banyaknya baris pada data
df.count()

1673

In [ ]:
# Melihat tipe data dari masing-masing kolom
df.dtypes

[('tanggal', 'string'),
 ('stasiun', 'string'),
 ('pm10', 'string'),
 ('so2', 'string'),
 ('co', 'string'),
 ('o3', 'string'),
 ('no2', 'string'),
 ('max', 'string'),
 ('critical', 'string'),
 ('categori', 'string')]

In [ ]:
# Mengganti tipe data menjadi tipe yang sesuai
df = df.withColumn("pm10", df["pm10"].cast('int'))
df = df.withColumn("so2", df["so2"].cast('int'))
df = df.withColumn("co", df["co"].cast('int'))
df = df.withColumn("o3", df["o3"].cast('int'))
df = df.withColumn("no2", df["no2"].cast('int'))
df = df.withColumn("max", df["max"].cast('int'))

In [ ]:
# Mengubah pyspark df ke pandas df
# Lalu, ekstrak kembali data yang sudah bersih ke csv untuk proses visualisasi di looker studio
df.toPandas().to_csv('Data_ISPU_clean.csv')

# **Analisis dan Visualisasi Data**

In [ ]:
# Melihat statistik dari masing-masing senyawa yang terkandung di udara
df.describe("pm10", "so2", "co", "o3", "no2", "max").show()

+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|summary|              pm10|               so2|                co|                o3|               no2|              max|
+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|  count|              1673|              1673|              1673|              1673|              1673|             1673|
|   mean| 49.09683203825463|23.160789001793187|21.457262402869098| 53.38971906754333|18.702331141661684|68.26300059772863|
| stddev|17.623017944980827| 16.21409813478605| 22.83052280802352|30.036778723048727| 25.86780620138816|23.14134175900994|
|    min|                 3|                 1|                 3|                 2|                 1|               17|
|    max|               111|               112|               197|               191|               197|              197|
+-------+-------

In [ ]:
# Analisis data dengan spark sql
# Membuat temporary tabel
df.createOrReplaceTempView("data_ISPU")

**Analisis 1**

In [ ]:
# (Analisis 1) Kualitas Udara Masing-masing Stasiun
a1 = spark.sql("""
  SELECT
    stasiun,
    categori,
    COUNT(categori)
  FROM data_ISPU
  GROUP BY stasiun, categori
  ORDER BY stasiun, categori
"""
)

# Menampilkan tabel: Kualitas Udara Masing-masing Stasiun
a1.show()

+--------------------+-----------+---------------+
|             stasiun|   categori|count(categori)|
+--------------------+-----------+---------------+
|  DKI1 (Bunderan HI)|       BAIK|            120|
|  DKI1 (Bunderan HI)|     SEDANG|            230|
|  DKI1 (Bunderan HI)|TIDAK SEHAT|              4|
|DKI2 (Kelapa Gading)|       BAIK|             75|
|DKI2 (Kelapa Gading)|     SEDANG|            249|
|DKI2 (Kelapa Gading)|TIDAK SEHAT|             31|
|    DKI3 (Jagakarsa)|       BAIK|             40|
|    DKI3 (Jagakarsa)|     SEDANG|            254|
|    DKI3 (Jagakarsa)|TIDAK SEHAT|             31|
| DKI4 (Lubang Buaya)|       BAIK|             46|
| DKI4 (Lubang Buaya)|     SEDANG|            252|
| DKI4 (Lubang Buaya)|TIDAK SEHAT|             21|
|  DKI5 (Kebon Jeruk)|       BAIK|             48|
|  DKI5 (Kebon Jeruk)|     SEDANG|            220|
|  DKI5 (Kebon Jeruk)|TIDAK SEHAT|             52|
+--------------------+-----------+---------------+



**Analisis 2**

In [ ]:
# (Analisis 2) Jumlah Nilai Indeks Polutan Setiap Stasiun
a2 = spark.sql("""
  SELECT
    stasiun,
    SUM(pm10) AS jumlah_pm10,
    SUM(so2) AS jumlah_so2,
    SUM(co) AS jumlah_co,
    SUM(o3) AS jumlah_o3,
    SUM(no2) AS jumlah_no2
  FROM data_ISPU
  GROUP BY stasiun
  ORDER BY jumlah_pm10 ASC
"""
)

# Menampilkan tabel: Jumlah Nilai Indeks Polutan Setiap Stasiun
a2.show()

+--------------------+-----------+----------+---------+---------+----------+
|             stasiun|jumlah_pm10|jumlah_so2|jumlah_co|jumlah_o3|jumlah_no2|
+--------------------+-----------+----------+---------+---------+----------+
|  DKI5 (Kebon Jeruk)|      14724|      5924|     8509|    20784|      6200|
|    DKI3 (Jagakarsa)|      15492|      8519|     8602|    17959|      5769|
|  DKI1 (Bunderan HI)|      15897|      7189|     4694|    14856|      6422|
|DKI2 (Kelapa Gading)|      17662|      8265|     5975|    19112|      7082|
| DKI4 (Lubang Buaya)|      18364|      8851|     8118|    16610|      5816|
+--------------------+-----------+----------+---------+---------+----------+



In [ ]:
a2.toPandas().to_csv('Data_Analisis_2.csv')

**Analisis 3**

In [ ]:
# (Analisis 3) Rata-rata Nilai Indeks Polutan Berdasarkan Tanggal

# Membuat tabel yang hanya terdiri dari kolom "tanggal", "pm10", "so2", "co", "o3", "no2"
a3 = spark.sql("""
  SELECT
    tanggal,
    pm10,
    so2,
    co,
    o3,
    no2
  FROM data_ISPU
"""
)

In [ ]:
import pandas as pd
a3_pd = a3.toPandas() # Mengubah hasil menjadi pandas dataframe
a3_pd["tanggal"] = pd.to_datetime(a3_pd["tanggal"]) # Mengubah tipe kolom "tanggal" dari object ke datetime

a3_pd["hari"] = a3_pd["tanggal"].dt.day # Menambahkan kolom hari
print(a3_pd)

        tanggal  pm10  so2  co  o3  no2  hari
0    2020-01-01    30   20  10  32    9     1
1    2020-02-01    27   22  12  29    8     1
2    2020-03-01    39   22  14  32   10     1
3    2020-04-01    34   22  14  38   10     1
4    2020-05-01    35   22  12  31    9     1
...         ...   ...  ...  ..  ..  ...   ...
1668 2020-12-24    29   31   9  28    2    24
1669 2020-12-25    24   27   7  18    3    25
1670 2020-12-28    22   33   5  35    3    28
1671 2020-12-30    16    7   3  21    2    30
1672 2020-12-31    18   13   6  24    3    31

[1673 rows x 7 columns]


<ipython-input-17-bd30645de367>:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  a3_pd["tanggal"] = pd.to_datetime(a3_pd["tanggal"]) # Mengubah tipe kolom "tanggal" dari object ke datetime


In [ ]:
a3_py = spark.createDataFrame(a3_pd)
a3_py.show()

# Membuat temporary tabel
a3_py.createOrReplaceTempView("data_analisis3")

+-------------------+----+---+---+---+---+----+
|            tanggal|pm10|so2| co| o3|no2|hari|
+-------------------+----+---+---+---+---+----+
|2020-01-01 00:00:00|  30| 20| 10| 32|  9|   1|
|2020-02-01 00:00:00|  27| 22| 12| 29|  8|   1|
|2020-03-01 00:00:00|  39| 22| 14| 32| 10|   1|
|2020-04-01 00:00:00|  34| 22| 14| 38| 10|   1|
|2020-05-01 00:00:00|  35| 22| 12| 31|  9|   1|
|2020-06-01 00:00:00|  46| 23| 16| 32|  9|   1|
|2020-07-01 00:00:00|  37| 23| 26| 33| 11|   1|
|2020-08-01 00:00:00|  41| 26| 20| 30| 11|   1|
|2020-09-01 00:00:00|  52| 23| 29| 24| 12|   1|
|2020-10-01 00:00:00|  24| 24| 18| 25|  8|   1|
|2020-11-01 00:00:00|  34| 31| 25| 23|  8|   1|
|2020-12-01 00:00:00|  27| 23|  9| 33|  4|   1|
|2020-01-13 00:00:00|  33| 26| 12| 36|  8|  13|
|2020-01-14 00:00:00|  34| 28| 13| 27|  7|  14|
|2020-01-15 00:00:00|  29| 22| 13| 36|  8|  15|
|2020-01-16 00:00:00|  52| 60| 19| 30|  8|  16|
|2020-01-17 00:00:00|  51| 34| 21| 74| 20|  17|
|2020-01-18 00:00:00|  37| 29| 14| 31|  

In [ ]:
# Membuat tabel: Rata-rata Nilai Indeks Polutan Berdasarkan Tanggal
a_3a = spark.sql("""
  SELECT
    hari,
    CAST(AVG(pm10) AS DECIMAL(5,2)) AS avg_pm10,
    CAST(AVG(so2) AS DECIMAL(5,2)) AS avg_so2,
    CAST(AVG(co) AS DECIMAL(5,2)) AS avg_co,
    CAST(AVG(o3) AS DECIMAL(5,2)) AS avg_o3,
    CAST(AVG(no2) AS DECIMAL(5,2)) AS avg_no2
  FROM data_analisis3
  GROUP BY hari
  ORDER BY hari ASC
"""
)

# Menampilkan tabel: Rata-rata Nilai Indeks Polutan Berdasarkan Tanggal
a_3a.show(31)

+----+--------+-------+------+------+-------+
|hari|avg_pm10|avg_so2|avg_co|avg_o3|avg_no2|
+----+--------+-------+------+------+-------+
|   1|   43.10|  20.44| 20.40| 45.28|   7.42|
|   2|   46.65|  15.75| 25.24| 47.80|   8.65|
|   3|   13.53|  21.98| 54.63|  8.37|  58.35|
|   4|   48.12|  14.00| 13.80| 61.55|   6.24|
|   5|   51.94|  13.75| 13.04| 56.50|   7.42|
|   6|   54.67|  15.36| 13.16| 60.00|   7.81|
|   7|   61.49|  19.59| 12.31| 65.81|   9.59|
|   8|   67.92|  20.34| 13.73| 74.41|   9.39|
|   9|   61.79|  24.00|  9.86| 83.40|   8.79|
|  10|   52.32|  19.14| 41.56| 52.90|  42.00|
|  11|   56.17|  61.81| 23.59| 38.14|  41.31|
|  12|   31.33|  33.02|  7.53| 33.61|   6.98|
|  13|   48.92|  23.38| 19.17| 53.43|  17.53|
|  14|   48.90|  25.88| 29.34| 51.44|  30.52|
|  15|   49.96|  22.65| 25.13| 51.00|  21.04|
|  16|   50.49|  24.27| 24.56| 49.24|  21.16|
|  17|   48.98|  24.07| 24.12| 59.16|  22.37|
|  18|   47.95|  23.10| 22.50| 56.17|  19.64|
|  19|   53.36|  24.39| 22.39| 60.

In [ ]:
a_3a.toPandas().to_csv('Data_Analisis_3.csv')

**Analisis 4**

In [ ]:
# (Analisis 4) Rata-rata Nilai Indeks Polutan Setiap Kategori

a4 = spark.sql("""
  SELECT
    categori,
    CAST(AVG(pm10+so2+co+o3+no2) AS DECIMAL(5,2)) AS avg_jmlh,
    CAST(AVG(pm10) AS DECIMAL(5,2)) AS avg_pm10,
    CAST(AVG(so2) AS DECIMAL(5,2)) AS avg_so2,
    CAST(AVG(co) AS DECIMAL(5,2)) AS avg_co,
    CAST(AVG(o3) AS DECIMAL(5,2)) AS avg_o3,
    CAST(AVG(no2) AS DECIMAL(5,2)) AS avg_no2
  FROM data_ISPU
  GROUP BY categori
  ORDER BY avg_jmlh ASC
"""
)

a4.show()

+-----------+--------+--------+-------+------+------+-------+
|   categori|avg_jmlh|avg_pm10|avg_so2|avg_co|avg_o3|avg_no2|
+-----------+--------+--------+-------+------+------+-------+
|       BAIK|  108.14|   30.96|  19.52| 15.12| 31.13|  11.41|
|     SEDANG|  172.19|   53.53|  23.80| 20.71| 56.00|  18.15|
|TIDAK SEHAT|  246.94|   53.57|  26.19| 42.96| 83.43|  40.79|
+-----------+--------+--------+-------+------+------+-------+



In [ ]:
a4.toPandas().to_csv('Data_Analisis_4.csv')

**Analisis 5**

In [ ]:
# (Analisis 5) Jenis Polutan Paling Kritis Setiap Bulan

a5 = spark.sql("""
  SELECT
    tanggal,
    pm10,
    so2,
    co,
    o3,
    no2,
    critical
  FROM data_ISPU
"""
)

a5.show(5)

+--------+----+---+---+---+---+--------+
| tanggal|pm10|so2| co| o3|no2|critical|
+--------+----+---+---+---+---+--------+
|1/1/2020|  30| 20| 10| 32|  9|      O3|
|2/1/2020|  27| 22| 12| 29|  8|      O3|
|3/1/2020|  39| 22| 14| 32| 10|    PM10|
|4/1/2020|  34| 22| 14| 38| 10|      O3|
|5/1/2020|  35| 22| 12| 31|  9|    PM10|
+--------+----+---+---+---+---+--------+
only showing top 5 rows



In [ ]:
import pandas as pd
a5_pd = a5.toPandas() # Mengubah hasil menjadi pandas dataframe
a5_pd["tanggal"] = pd.to_datetime(a5_pd["tanggal"]) # Mengubah tipe kolom data dari object ke datetime

a5_pd["bulan"] = a5_pd["tanggal"].dt.month
print(a5_pd)

        tanggal  pm10  so2  co  o3  no2 critical  bulan
0    2020-01-01    30   20  10  32    9       O3      1
1    2020-02-01    27   22  12  29    8       O3      2
2    2020-03-01    39   22  14  32   10     PM10      3
3    2020-04-01    34   22  14  38   10       O3      4
4    2020-05-01    35   22  12  31    9     PM10      5
...         ...   ...  ...  ..  ..  ...      ...    ...
1668 2020-12-24    29   31   9  28    2      SO2     12
1669 2020-12-25    24   27   7  18    3      SO2     12
1670 2020-12-28    22   33   5  35    3       O3     12
1671 2020-12-30    16    7   3  21    2       O3     12
1672 2020-12-31    18   13   6  24    3       O3     12

[1673 rows x 8 columns]


<ipython-input-24-3149a760ad48>:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  a5_pd["tanggal"] = pd.to_datetime(a5_pd["tanggal"]) # Mengubah tipe kolom data dari object ke datetime


In [ ]:
a5_py = spark.createDataFrame(a5_pd)
a5_py.show()

# Membuat temporary tabel
a5_py.createOrReplaceTempView("data_analisis5")

+-------------------+----+---+---+---+---+--------+-----+
|            tanggal|pm10|so2| co| o3|no2|critical|bulan|
+-------------------+----+---+---+---+---+--------+-----+
|2020-01-01 00:00:00|  30| 20| 10| 32|  9|      O3|    1|
|2020-02-01 00:00:00|  27| 22| 12| 29|  8|      O3|    2|
|2020-03-01 00:00:00|  39| 22| 14| 32| 10|    PM10|    3|
|2020-04-01 00:00:00|  34| 22| 14| 38| 10|      O3|    4|
|2020-05-01 00:00:00|  35| 22| 12| 31|  9|    PM10|    5|
|2020-06-01 00:00:00|  46| 23| 16| 32|  9|    PM10|    6|
|2020-07-01 00:00:00|  37| 23| 26| 33| 11|    PM10|    7|
|2020-08-01 00:00:00|  41| 26| 20| 30| 11|    PM10|    8|
|2020-09-01 00:00:00|  52| 23| 29| 24| 12|    PM10|    9|
|2020-10-01 00:00:00|  24| 24| 18| 25|  8|      O3|   10|
|2020-11-01 00:00:00|  34| 31| 25| 23|  8|    PM10|   11|
|2020-12-01 00:00:00|  27| 23|  9| 33|  4|      O3|   12|
|2020-01-13 00:00:00|  33| 26| 12| 36|  8|      O3|    1|
|2020-01-14 00:00:00|  34| 28| 13| 27|  7|    PM10|    1|
|2020-01-15 00

In [ ]:
a5_a = spark.sql("""
  SELECT
    bulan,
    MAX(jumlah_critical) AS max_jumlah_critical
  FROM (
    SELECT
      bulan,
      critical,
      COUNT(critical) AS jumlah_critical
    FROM data_analisis5
    GROUP BY bulan,critical
  )
  GROUP BY bulan
  ORDER BY bulan
"""
)
a5_a.show()

+-----+-------------------+
|bulan|max_jumlah_critical|
+-----+-------------------+
|    1|                 70|
|    2|                 75|
|    3|                 69|
|    4|                 78|
|    5|                 92|
|    6|                 70|
|    7|                 76|
|    8|                 75|
|    9|                 96|
|   10|                 75|
|   11|                 48|
|   12|                 73|
+-----+-------------------+



In [ ]:
a5_b = spark.sql("""
  SELECT
    bulan,
    critical,
    MAX(jumlah_critical) AS max_jumlah_critical
  FROM (
    SELECT
      bulan,
      critical,
      COUNT(critical) AS jumlah_critical
    FROM data_analisis5
    GROUP BY bulan,critical
  )
  GROUP BY bulan, critical
  HAVING max_jumlah_critical IN(70,75,69,78,92,76,96,48,73)
  ORDER BY bulan
"""
)

a5_b.show()

+-----+--------+-------------------+
|bulan|critical|max_jumlah_critical|
+-----+--------+-------------------+
|    1|      O3|                 70|
|    2|      O3|                 75|
|    3|      CO|                 69|
|    4|      O3|                 78|
|    5|      O3|                 92|
|    6|      O3|                 70|
|    7|      O3|                 76|
|    8|      O3|                 75|
|    9|      O3|                 96|
|   10|      O3|                 75|
|   11|     SO2|                 48|
|   11|      O3|                 48|
|   12|      O3|                 73|
+-----+--------+-------------------+



In [ ]:
a5_b.toPandas().to_csv('Data_Analisis_5.csv')